In [1]:
#q1b
import numpy as np
from scipy.optimize import minimize
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt


def dualClassify(Xtest, a, b, X, y, lamb, kernel_func):

    m = Xtest.shape[0]
    K_test_train = kernel_func(Xtest, X)
    delta_y_a = y * a
    # (1 / lamb) * K_test_train @ delta_y_a + b
    predictions = (1 / lamb) * (K_test_train @ delta_y_a) + b
    
    yhat = np.sign(predictions)
    
    return yhat





In [27]:

import pandas as pd

def split_arr(arr,n, k):
    #print(arr)
    return np.array_split(arr, k)  # Split into k groups


def cvMnist(dataset_folder, lamb_list, kernel_list, k=5):


    #train_data = pd.read_csv(f"{dataset_folder}/A2train.csv", header=None).to_numpy()
    train_data =np.random.randint(0, 10, size=(10, 7))

    print("EVERYTHING",train_data)

    X = train_data[:, 1:] #/ 255.
    y = train_data[:, 0][:, None]
    y[y == 4] = -1
    y[y == 9] = 1

    print("Data:",X)
    print("Labels",y)
    
    #cv_acc = np.zeros([k, len(lamb_list), len(kernel_list)]) 

    # : perform any necessary setup Kfolds

    split_data = split_arr(X,X.shape[0],k)
    print("Split Data: " , split_data)
    split_labels = split_arr(y,y.shape[0],k)
    print("Split Labels: " , split_labels)

    #print(X)
    #print(y)
    


    for i, lamb in enumerate(lamb_list):
        for j, kernel_func in enumerate(kernel_list):
            for l in range(k):
                Xtrain =  # : obtain the training input
                ytrain = # : obtain the corresponding training label
                Xval = # : obtain the validation input
                yval = # : obtain the corresponding validation label
                a, b = dualHinge(Xtrain, ytrain, lamb, kernel_func)
                yhat = dualClassify(Xval, a, b, Xtrain, ytrain, lamb, kernel_func) 
                cv_acc[l, i, j] = #  calculate validation accuracy
    
    #  compute the average accuracies over k folds
    #  identify the best lamb and kernel function
    #  return a "len(lamb_list)-by-len(kernel_list)" accuracy variable,
    #       the best lamb and the best kernel 


dataset_folder = '/Users/dantefarinon-spezzano/Downloads/A2files'

cvMnist(dataset_folder,None,None)

EVERYTHING [[0 0 5 9 6 9 1]
 [9 3 6 3 7 4 2]
 [8 0 6 2 7 5 3]
 [7 4 3 7 5 8 2]
 [4 2 3 0 9 7 0]
 [9 8 6 0 2 3 2]
 [0 8 0 0 5 9 4]
 [5 9 4 0 8 5 4]
 [9 3 8 7 1 5 3]
 [2 2 9 0 4 5 8]]
Data: [[0 5 9 6 9 1]
 [3 6 3 7 4 2]
 [0 6 2 7 5 3]
 [4 3 7 5 8 2]
 [2 3 0 9 7 0]
 [8 6 0 2 3 2]
 [8 0 0 5 9 4]
 [9 4 0 8 5 4]
 [3 8 7 1 5 3]
 [2 9 0 4 5 8]]
Labels [[ 0]
 [ 1]
 [ 8]
 [ 7]
 [-1]
 [ 1]
 [ 0]
 [ 5]
 [ 1]
 [ 2]]
Split Data:  [array([[0, 5, 9, 6, 9, 1],
       [3, 6, 3, 7, 4, 2]]), array([[0, 6, 2, 7, 5, 3],
       [4, 3, 7, 5, 8, 2]]), array([[2, 3, 0, 9, 7, 0],
       [8, 6, 0, 2, 3, 2]]), array([[8, 0, 0, 5, 9, 4],
       [9, 4, 0, 8, 5, 4]]), array([[3, 8, 7, 1, 5, 3],
       [2, 9, 0, 4, 5, 8]])]
Split Labels:  [array([[0],
       [1]]), array([[8],
       [7]]), array([[-1],
       [ 1]]), array([[0],
       [5]]), array([[1],
       [2]])]


In [8]:
import numpy as np

# Generate an array of 100 elements
arr = np.arange(100)
print(arr)

# Define function to split indices into k groups
def split_indices(arr, n, k):
    print(arr)
    return np.array_split(arr, k)  # Split into k groups

# Split into 5 groups
splits = split_indices(arr, len(arr), 5)

# Print the split groups
for i, group in enumerate(splits):
    print(f"Group {i+1}: {group}")


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Group 1: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Group 2: [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
Group 3: [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59]
Group 4: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79]
Group 5: [80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99]
